<a href="https://colab.research.google.com/github/goddess1992/federated_new/blob/master/federated_yolov5_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AlanSunAlan/federated_train_yolov5.git

Cloning into 'federated_train_yolov5'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 46 (delta 25), reused 19 (delta 6), pack-reused 0
Receiving objects: 100% (46/46), 15.26 KiB | 3.05 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [ ]:
!cd federated_train_yolov5 && git clone https://github.com/ultralytics/yolov5.git

In [3]:
!cd federated_train_yolov5/yolov5 && pip install -r requirements.txt

In [ ]:
!python federated_train_yolov5/yolov5/train.py

In [ ]:
!python federated_train_yolov5/centralized_train.py

In [ ]:
# !wget https://ultralytics.com/assets/coco128.zip
!unzip "coco128.zip" -d "/content/federated_train_yolov5/datasets/"


In [58]:
import sys, os
import shutil
import yaml

ROOT = "/content/federated_train_yolov5"
DATASETS = os.path.join("/content/federated_train_yolov5/datasets")
VOC_ROOT = os.path.join(DATASETS, "coco128")
TRAIN_IMG = os.path.join(VOC_ROOT, "merge_images")
TRAIN_LABEL = os.path.join(VOC_ROOT, "merge_labels")

YAML_ROOT = os.path.join(ROOT, "data")


NUM_CLIENTS = 4

def mergeTrainVal():

    orig_train_img = os.path.join(VOC_ROOT, "images", "train2017")
    orig_train_lb = os.path.join(VOC_ROOT, "labels", "train2017")
    orig_val_img = os.path.join(VOC_ROOT, "images", "train2017")
    orig_val_lb = os.path.join(VOC_ROOT, "labels", "train2017")

    dst_merge_img = os.path.join(VOC_ROOT, "merge_images")
    dst_merge_lb = os.path.join(VOC_ROOT, "merge_labels")

    if os.path.isdir(dst_merge_img) or os.path.isdir(dst_merge_lb):
        shutil.rmtree(dst_merge_img)
        shutil.rmtree(dst_merge_lb)

    os.mkdir(dst_merge_img)
    os.mkdir(dst_merge_lb)

    train_img_ids = []
    for train_img_name in os.listdir(orig_train_img):
        train_img_ids.append(train_img_name.split('.')[0])
    val_img_ids = []
    for val_img_name in os.listdir(orig_val_img):
        val_img_ids.append(val_img_name.split('.')[0])

    for train_img_id in train_img_ids:
        src_img = os.path.join(orig_train_img, train_img_id + '.jpg')
        src_lb = os.path.join(orig_train_lb, train_img_id + '.txt')

        dst_img = os.path.join(dst_merge_img, train_img_id + '.jpg')
        dst_lb = os.path.join(dst_merge_lb, train_img_id + '.txt')

        shutil.copyfile(src_img, dst_img)
        shutil.copyfile(src_lb, dst_lb)

    for val_img_id in val_img_ids:
        src_img = os.path.join(orig_val_img, val_img_id + '.jpg')
        src_lb = os.path.join(orig_val_lb, val_img_id + '.txt')

        dst_img = os.path.join(dst_merge_img, val_img_id + '.jpg')
        dst_lb = os.path.join(dst_merge_lb, val_img_id + '.txt')

        shutil.copyfile(src_img, dst_img)
        shutil.copyfile(src_lb, dst_lb)

def saveYaml(data_folder):
    save_path = os.path.join(YAML_ROOT, data_folder+".yaml")

    output = {"path": "../datasets/{}".format(data_folder)}
    output["train"] = "images"
    output["val"] = "../coco128/images/test2017"
    output["val"] = "../coco128/images/test2017"

    output['nc'] = 80
    output['names'] = ['person','bicycle','car','motorcycle','airplane','bus','train','truck','boat','traffic light','fire hydrant','stop sign','parking meter','bench','bird','cat','dog','horse','sheep','cow','elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase','frisbee','skis','snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard','tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich','orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','couch','potted plant','bed','dining table','toilet','tv','laptop','mouse','remote','keyboard','cell phone','microwave','oven','toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear','hair drier','toothbrush']

    with open(save_path, "w+") as f:
        yaml.dump(output, f)


def copyData(img_id, client_root_folder):
    img_file_name = str(img_id) + ".jpg"
    label_file_name = str(img_id) + ".txt"

    src_img_path = os.path.join(TRAIN_IMG, img_file_name)
    src_label_path = os.path.join(TRAIN_LABEL, label_file_name)

    client_img_folder = os.path.join(client_root_folder, "images", img_file_name)
    client_label_folder = os.path.join(client_root_folder, "labels", label_file_name)

    shutil.copyfile(src_img_path, client_img_folder)
    shutil.copyfile(src_label_path, client_label_folder)

def evenSplit(output_prefix="even_"):
    # Get names without suffix
    img_files = os.listdir(TRAIN_IMG)

    img_ids = []
    for img_file in img_files:
        img_id = img_file.split('.')[0]
        img_ids.append(img_id)
    img_ids.sort()

    # Make directories for each client
    for client_id in range(0, NUM_CLIENTS):
        folder_name = "{}{}".format(output_prefix, str(client_id))
        folder_path = os.path.join("/content/federated_train_yolov5/datasets", folder_name,"coco128")
        # For each client, create a folder
        if os.path.isdir(folder_path):
            # Already exits, delete the whole folder
            shutil.rmtree(folder_path)
        # Create a folder for each client
        os.mkdir(folder_path)
        # Create image and label folders
        image_folder_path = os.path.join(folder_path, "images")
        label_folder_path = os.path.join(folder_path, "labels")
        os.mkdir(image_folder_path)
        os.mkdir(label_folder_path)


    current_client = 0
    # Copy image from VOC to the client folders
    for img_id in img_ids:
        if current_client == NUM_CLIENTS - 1:
            current_client = 0
        else:
            current_client += 1

        folder_name = "{}{}".format(output_prefix, str(current_client))
        folder_path = os.path.join(DATASETS, folder_name)
        copyData(img_id, folder_path)

        print("Copying {} to {}".format(img_id, folder_path))

    # Save yaml file
    for client_id in range(0, NUM_CLIENTS):
        saveYaml("{}{}".format(output_prefix, str(client_id)))
        print("Yaml file saved for ", client_id)


def noniidSplit(output_prefix="noniid_", split_num=4, split_ratios=[0.1, 0.2, 0.3, 0.4]):
    assert len(split_ratios) == split_num
    assert sum(split_ratios) == 1


    # Get names without suffix
    img_files = os.listdir(TRAIN_IMG)

    img_ids = []
    for img_file in img_files:
        img_id = img_file.split('.')[0]
        img_ids.append(img_id)
    img_ids.sort()

    # Make directories for each client
    for client_id in range(0, NUM_CLIENTS):
        folder_name = "{}{}".format(output_prefix, str(client_id))
        folder_path = os.path.join(DATASETS, folder_name)
        # For each client, create a folder
        if os.path.isdir(folder_path):
            # Already exits, delete the whole folder
            shutil.rmtree(folder_path)
        # Create a folder for each client
        os.mkdir(folder_path)
        # Create image and label folders
        image_folder_path = os.path.join(folder_path, "images")
        label_folder_path = os.path.join(folder_path, "labels")
        os.mkdir(image_folder_path)
        os.mkdir(label_folder_path)

    # Calculate the splitting indexes
    split_indices = [] # In form of [[start1, end1], [start2, end2], ...]
    for client_i, split_ratio in enumerate(split_ratios):
        if client_i == 0:
            start_index = 0
        else:
            start_index = split_indices[client_i -1][-1] + 1

        dataset_size = len(img_ids)
        sub_size = int(dataset_size * split_ratio)


        if client_i == split_num - 1:
            end_index = dataset_size - 1
        else:
            end_index = sub_size + start_index

        split_indices.append([start_index, end_index])

    # Copy image from VOC to the client folders
    for client_id, split_index in enumerate(split_indices):
        for img_index in range(split_index[0], split_index[1]):
            img_id = img_ids[img_index]
            folder_name = "{}{}".format(output_prefix, str(client_id))
            folder_path = os.path.join(DATASETS, folder_name)
            copyData(img_id, folder_path)

            print("Copying {} to {}".format(img_id, folder_path))

    # Save yaml file
    for client_id in range(0, NUM_CLIENTS):
        saveYaml("{}{}".format(output_prefix, str(client_id)))
        print("Yaml file saved for ", client_id)

if __name__ == "__main__":
    mergeTrainVal()
    evenSplit()
    noniidSplit()


Copying 000000000009 to /content/federated_train_yolov5/datasets/even_1
Copying 000000000025 to /content/federated_train_yolov5/datasets/even_2
Copying 000000000030 to /content/federated_train_yolov5/datasets/even_3
Copying 000000000034 to /content/federated_train_yolov5/datasets/even_0
Copying 000000000036 to /content/federated_train_yolov5/datasets/even_1
Copying 000000000042 to /content/federated_train_yolov5/datasets/even_2
Copying 000000000049 to /content/federated_train_yolov5/datasets/even_3
Copying 000000000061 to /content/federated_train_yolov5/datasets/even_0
Copying 000000000064 to /content/federated_train_yolov5/datasets/even_1
Copying 000000000071 to /content/federated_train_yolov5/datasets/even_2
Copying 000000000072 to /content/federated_train_yolov5/datasets/even_3
Copying 000000000073 to /content/federated_train_yolov5/datasets/even_0
Copying 000000000074 to /content/federated_train_yolov5/datasets/even_1
Copying 000000000077 to /content/federated_train_yolov5/datasets

In [61]:
!python federated_train_yolov5/federated_train_even.py

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

**1**
**2**
**3**
/content/federated_train_yolov5/data/even_0.yaml
/content/federated_train_yolov5/training/federated_even_distributed/0
/content/federated_train_yolov5/data/even_1.yaml
/content/federated_train_yolov5/training/federated_even_distributed/1
/content/federated_train_yolov5/data/even_2.yaml
/content/federated_train_yolov5/training/federated_even_distributed/2
/content/federated_train_yolov5/data/even_3.yaml
/content/federated_train_yolov5/training/federa

In [63]:
!rsync -r --progress "./federated_train_yolov5" "/content/drive/MyDrive/"

sending incremental file list
federated_train_yolov5/
federated_train_yolov5/README.md
          1,285 100%    0.00kB/s    0:00:00 (xfr#1, ir-chk=1043/1045)
federated_train_yolov5/centralized_train.py
          4,339 100%    4.14MB/s    0:00:00 (xfr#2, ir-chk=1042/1045)
federated_train_yolov5/federated_train_even.py
          3,236 100%    3.09MB/s    0:00:00 (xfr#3, ir-chk=1041/1045)
federated_train_yolov5/federated_train_noniid.py
          4,293 100%    4.09MB/s    0:00:00 (xfr#4, ir-chk=1040/1045)
federated_train_yolov5/split_voc2007.py
          7,736 100%    7.38MB/s    0:00:00 (xfr#5, ir-chk=1039/1045)
federated_train_yolov5/yolov5s.pt
     14,808,437 100%   75.12MB/s    0:00:00 (xfr#6, ir-chk=1038/1045)
federated_train_yolov5/.git/
federated_train_yolov5/.git/HEAD
             21 100%    0.05kB/s    0:00:00 (xfr#7, ir-chk=1031/1045)
federated_train_yolov5/.git/config
            278 100%    0.66kB/s    0:00:00 (xfr#8, ir-chk=1030/1045)
federated_train_yolov5/.git/description
  